# Quickstart for Text Analytics API with Python 
<a name="HOLTop"></a>

Use this quickstart to begin analyzing language with the Text Analytics SDK for Python. [Text Analytics](//go.microsoft.com/fwlink/?LinkID=759711) REST API is compatible with most programming languages, the SDK provides an easy way to integrate the service into your applications.

Refer to the [API definitions](//go.microsoft.com/fwlink/?LinkID=759346) for technical documentation for the APIs.

You can run this example as a Jupyter notebook on [MyBinder](https://mybinder.org) by clicking on the launch Binder badge: 

[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/Microsoft/cognitive-services-notebooks/master?filepath=TextAnalytics.ipynb)


## Prerequisites

You must have a [Cognitive Services API account](https://docs.microsoft.com/azure/cognitive-services/cognitive-services-apis-create-account) with **Text Analytics API**. You can use the **free tier for 5,000 transactions/month** to complete this walkthrough.

You must also have the [endpoint and access key](../How-tos/text-analytics-how-to-access-key.md) that was generated for you during sign-up. 

To continue with this walkthrough, replace `subscription_key` with a valid subscription key that you obtained earlier.

In [2]:
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials
import json
from pprint import pprint

subscription_key = None
assert subscription_key
credentials = CognitiveServicesCredentials(subscription_key)

Next, verify that the region in `text_analytics_base_url` corresponds to the one you used when setting up the service. If you are using a free trial key, you do not need to change anything.

In [23]:
text_analytics_url = "https://westcentralus.api.cognitive.microsoft.com/"
text_analytics = TextAnalyticsClient(endpoint=text_analytics_url, credentials=credentials)

<a name="SentimentAnalysis"></a>

## Analyze sentiment


1. Create a list of dictionaries, each representing a document you want to analyze.

In [88]:
documents = [
  {"id": "1", "language": "en", "text": "I had the best day of my life."},
  {"id": "2", "language": "en", "text": "This was a waste of my time. The speaker put me to sleep."},  
  {"id": "3", "language": "es", "text": "No tengo dinero ni nada que dar..."},  
  {"id": "4", "language": "it", "text": "L'hotel veneziano era meraviglioso. È un bellissimo pezzo di architettura."}
]

2. Call the `sentiment()` function and get the result. Then iterate through the results, and print each document's ID, and sentiment score. A score closer to 0 indicates a negative sentiment, while a score closer to 1 indicates a positive sentiment.

In [91]:
response = text_analytics.sentiment(documents=documents)
for document in response.documents:
     print("Document Id: ", document.id, ", Sentiment Score: ", "{:.2f}".format(document.score))

Document Id:  1 , Sentiment Score:  0.87
Document Id:  2 , Sentiment Score:  0.11
Document Id:  3 , Sentiment Score:  0.44
Document Id:  4 , Sentiment Score:  1.00


<a name="Detect"></a>

## Detect languages


1. Create a list of dictionaries, containing the documents you want to analyze.

In [93]:
documents = [
    { 'id': '1', 'text': 'This is a document written in English.' },
    { 'id': '2', 'text': 'Este es un document escrito en Español.' },
    { 'id': '3', 'text': '这是一个用中文写的文件' }
]

2. Using the client created earlier, call `detect_language()` function and get the result. Then iterate through the results, and print each document's ID, and the first returned language.

In [94]:
response = text_analytics.detect_language(documents=documents)
for document in response.documents:
    print("Document Id: ", document.id , ", Language: ", document.detected_languages[0].name)

Document Id:  1 , Language:  English
Document Id:  2 , Language:  Spanish
Document Id:  3 , Language:  Chinese_Simplified


The following lines of code render the response object as an HTML table.

In [54]:
from IPython.display import HTML
table = []
for document in response.documents:
    text  = next(filter(lambda d: d["id"] == document.id, documents))["text"]
    langs = ", ".join(["{0}({1})".format(lang.name, lang.score) for lang in document.detected_languages])
    table.append("<tr><td>{0}</td><td>{1}</td>".format(text, langs))
HTML("<table><tr><th>Text</th><th>Detected languages(scores)</th></tr>{0}</table>".format("\n".join(table)))

Text,Detected languages(scores)
This is a document written in English.,English(1.0)
Este es un document escrito en Español.,Spanish(1.0)
这是一个用中文写的文件,Chinese_Simplified(1.0)


## Entity Extraction

1. Create a list of dictionaries, containing the documents you want to analyze.

In [110]:
documents = [
    {"id": "1","language": "en", "text": "Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975, to develop and sell BASIC interpreters for the Altair 8800."},
    {"id": "2","language": "es", "text": "La sede principal de Microsoft se encuentra en la ciudad de Redmond, a 21 kilómetros de Seattle."}
]

2.  Using the client created earlier, call `entities()` function and get the result. Then iterate through the results, and print each document's ID, and the entities contained in it.

In [111]:
response = text_analytics.entities(documents=documents)

for document in response.documents:
    print("Document Id: ", document.id)
    print("\tKey Entities:")
    for entity in document.entities:
        print("\t\t", "NAME: ",entity.name, "\tType: ", entity.type, "\tSub-type: ", entity.sub_type)
        for match in entity.matches:
            print("\t\t\tOffset: ", match.offset, "\tLength: ", match.length, "\tScore: ",
                  "{:.2f}".format(match.entity_type_score))

Document Id:  1
	Key Entities:
		 NAME:  Microsoft 	Type:  Organization 	Sub-type:  None
			Offset:  0 	Length:  9 	Score:  1.00
		 NAME:  Bill Gates 	Type:  Person 	Sub-type:  None
			Offset:  25 	Length:  10 	Score:  1.00
		 NAME:  Paul Allen 	Type:  Person 	Sub-type:  None
			Offset:  40 	Length:  10 	Score:  1.00
		 NAME:  April 4 	Type:  Other 	Sub-type:  None
			Offset:  54 	Length:  7 	Score:  0.80
		 NAME:  April 4, 1975 	Type:  DateTime 	Sub-type:  Date
			Offset:  54 	Length:  13 	Score:  0.80
		 NAME:  BASIC 	Type:  Other 	Sub-type:  None
			Offset:  89 	Length:  5 	Score:  0.80
		 NAME:  Altair 8800 	Type:  Other 	Sub-type:  None
			Offset:  116 	Length:  11 	Score:  0.80
Document Id:  2
	Key Entities:
		 NAME:  Microsoft 	Type:  Organization 	Sub-type:  None
			Offset:  21 	Length:  9 	Score:  1.00
		 NAME:  Redmond (Washington) 	Type:  Location 	Sub-type:  None
			Offset:  60 	Length:  7 	Score:  0.99
		 NAME:  21 kilómetros 	Type:  Quantity 	Sub-type:  Dimension
			Offse

<a name="KeyPhraseExtraction"></a>

## Key phrase extraction


1. Create a list of dictionaries, containing the documents you want to analyze.

In [95]:
documents = [
    {"id": "1", "language": "ja", "text": "猫は幸せ"},
    {"id": "2", "language": "de", "text": "Fahrt nach Stuttgart und dann zum Hotel zu Fu."},
    {"id": "3", "language": "en", "text": "My cat might need to see a veterinarian."},
    {"id": "4", "language": "es", "text": "A mi me encanta el fútbol!"}
            ]

2. Using the client created earlier, call `key_phrases()` function and get the result. Then iterate through the results, and print each document's ID, and the key phrases contained in it.

In [98]:
response = text_analytics.key_phrases(documents=documents)

for document in response.documents:
    print("Document Id: ", document.id)
    print("\tKey Phrases:")
    for phrase in document.key_phrases:
        print("\t\t",phrase)


Document Id:  1
	Phrases:
		 幸せ
Document Id:  2
	Phrases:
		 Stuttgart
		 Hotel
		 Fahrt
		 Fu
Document Id:  3
	Phrases:
		 cat
		 veterinarian
Document Id:  4
	Phrases:
		 fútbol


The JSON object can once again be rendered as an HTML table using the following lines of code:

In [85]:
from IPython.display import HTML
table = []
for document in response.documents:
    text    = next(filter(lambda d: d["id"] == document.id, documents))["text"]    
    phrases = ",".join(document.key_phrases)
    table.append("<tr><td>{0}</td><td>{1}</td>".format(text, phrases))
HTML("<table><tr><th>Text</th><th>Key phrases</th></tr>{0}</table>".format("\n".join(table)))

Text,Key phrases
I had a wonderful experience! The rooms were wonderful and the staff was helpful.,"wonderful experience,staff,rooms"
I had a terrible time at the hotel. The staff was rude and the food was awful.,"food,terrible time,hotel,staff"
Los caminos que llevan hasta Monte Rainier son espectaculares y hermosos.,"Monte Rainier,caminos"
La carretera estaba atascada. Había mucho tráfico el día de ayer.,"carretera,tráfico,día"


## Next steps

> [!div class="nextstepaction"]
> [Text Analytics With Power BI](../tutorials/tutorial-power-bi-key-phrases.md)

## See also 

 [Text Analytics overview](../overview.md)  
 [Frequently asked questions (FAQ)](../text-analytics-resource-faq.md)